In [1]:
import sys

sys.path.insert(0, '../')

from modules.spec_dataset import *
from modules.train_prep import *
from modules.plot_results import *

In [2]:
class SpectroMLP(nn.Module):
    """
    Defines a multilayer perceptron (MLP) for classification of spectrogram-based data.
    
    Parameters:
    - input_size (int): The number of features in the input data.
    - num_classes (int): The number of classes for classification.

    The network consists of two hidden layers and an output layer.
    """
    def __init__(self, input_size, num_classes=4):
        super(SpectroMLP, self).__init__()
        # Initialize the parent class, nn.Module.
        
        # Define the first hidden layer. This layer takes the input and transforms it to a 512-dimensional space.
        self.fc1 = nn.Linear(input_size, 512)  # First hidden layer
        
        # Define the second hidden layer. This layer further transforms the representation to a 256-dimensional space.
        self.fc2 = nn.Linear(512, 256)         # Second hidden layer
        
        # Define the output layer. This layer maps the features from the second hidden layer to the class scores.
        self.fc3 = nn.Linear(256, num_classes) # Output layer, which will output 'num_classes' scores.

    def forward(self, x):
        """
        Defines the forward pass of the network.

        Parameters:
        - x (Tensor): The input data tensor.

        Returns:
        - x (Tensor): The output tensor containing class scores for each input sample.
        """
        # Flatten the input tensor if not already flattened. This is necessary because
        # nn.Linear expects its input to be of shape (batch_size, num_features).
        x = x.view(x.size(0), -1)
        
        # Pass the input through the first hidden layer and apply the ReLU activation function.
        x = F.relu(self.fc1(x))  # Activation function for the first layer
        
        # Pass the output of the first layer through the second hidden layer and apply the ReLU activation function.
        x = F.relu(self.fc2(x))  # Activation function for the second layer
        
        # Pass the output of the second layer through the output layer. Note that no activation function
        # is applied here, which is common for classification tasks where the raw scores are used
        # with a softmax function during the loss computation.
        x = self.fc3(x)          # No activation function applied at the output layer

        return x


In [ ]:
# Check if CUDA is available on the device and set the CUDA device accordingly
if torch.cuda.is_available():
    torch.cuda.set_device(cuda_device)  # Assuming 'cuda_device' is defined in 'modules.constants'
elif torch.backends.mps.is_available():
    mps_device = torch.device("mps")  # Set device to use Metal Performance Shaders on macOS

# Define data transformations, which include converting images or tensors to PyTorch tensors
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),  # Transform the data to a tensor, normalizing the values
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
    ]),
}

# Setup datasets for training and validation using the custom SpectroDataset class
dsets = {
    'train': SpectroDataset(train_dir, train_behav_file, data_transforms['train']),
    'val': SpectroDataset(val_dir, val_behav_file, data_transforms['val'])
}
dset_sizes = {split: len(dsets[split]) for split in ['train', 'val']}  # Calculate the size of each dataset

# Create data loaders that handle sampling and provide batches of data to the model during training
dset_loaders = {}
for split in ['train', 'val']:
    # Calculate class weights for handling imbalanced data
    targets = np.array([dsets[split].get_label(i) for i in range(len(dsets[split]))])
    class_counts = dsets[split].get_class_counts()
    class_weights = np.array([1.0 / class_counts[label] if class_counts[label] > 0 else 0 for label in targets])
    sampler = WeightedRandomSampler(class_weights, num_samples=len(class_weights), replacement=True)
    dset_loaders[split] = torch.utils.data.DataLoader(dsets[split], batch_size=b_size, num_workers=0, sampler=sampler)
    print('done making loader:', split)  # Print a message once loaders are ready

# Initialize the neural network model, the loss function, and the optimizer
model_ft = SpectroMLP(input_size=108360, num_classes=4)  # 108360 should match the flattened input size of your data
criterion = nn.CrossEntropyLoss()  # Standard loss function for classification tasks
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.0001, weight_decay=1e-5)  # Adam optimizer with specified learning rate and weight decay

# Configure the device on which the model will run
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
model_ft.to(device)  # Move the model to the configured device
criterion.to(device)  # Move the criterion to the configured device
print(f"Training on {device}")  # Inform the user of the device being used

# Train the model using a custom training function that needs to be defined elsewhere in your codebase
model_ft, accuracies, losses, preds, labels = train_model(
    model_ft, criterion, optimizer_ft, exp_lr_scheduler, dset_loaders, dset_sizes, num_epochs=n_epochs
)


In [ ]:
# Loop through each dataset split type, typically 'train' and 'val' for training and validation phases.
for split in ['train', 'val']:
    # Print out the accuracy for each epoch during training and validation.
    # 'accuracies' is a dictionary where keys are the split types and values are lists of accuracy values per epoch.
    print(split, 'accuracies by epoch:', accuracies[split])
    
    # Similarly, print out the loss for each epoch during training and validation.
    # 'losses' is similarly structured with lists of loss values per epoch.
    print(split, 'losses by epoch:', losses[split])

# Save the trained model's state dictionary. This is the set of model parameters (weights and biases) learned during training.
torch.save(model_ft.state_dict(), '../../models/MLP_spec_best_model.pt')

# Plot the training history using the updated accuracy and loss data.
# This function likely generates graphs showing how accuracy and loss changed over the course of training.
plot_training_history(accuracies, losses, 'MLP_spec')

# Plot a confusion matrix using the true labels and predicted labels from the model.
# This visualization helps to understand how well the model is performing across different classes,
# showing where it makes correct predictions and where it confuses one class for another.
plot_confusion_matrix(labels, preds, 'MLP_spec')